In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
from pyspark.sql.types import MapType, IntegerType, DoubleType, StringType, StructType, StructField
import pyspark.sql.functions as f

schema = StructType([
            StructField('column1', IntegerType()),
            StructField('column2', IntegerType()),
            StructField('column3', MapType(StringType(), DoubleType()))])

data = [(1, 2, {'a':3.5, 'b':4.2}), (4, 8, {'b':3.7, 'e':4.9})]
df = spark.createDataFrame(data, schema=schema)
df.show()

+-------+-------+--------------------+
column1|column2| column3|
+-------+-------+--------------------+
 1| 2|[a -> 3.5, b -> 4.2]|
 4| 8|[e -> 4.9, b -> 3.7]|
+-------+-------+--------------------+

In [3]:
my_dict = {'b':3.7, 'e':4.9}

' and '.join(["column3.{0}=map.{0}".format(x) for x in my_dict.keys()])


Out[37]: 'column3.b=map.b and column3.e=map.e'

In [5]:
df.show() #sample dataframe
my_dict = {'b':3.7, 'e':4.9} #dictionary to filter with

#+-------+-------+--------------------+
#|column1|column2|             column3|
#+-------+-------+--------------------+
#|      1|      2|[a -> 3.5, b -> 4.2]|
#|      4|      8|[e -> 4.9, b -> 3.7]|
#+-------+-------+--------------------+

from pyspark.sql import functions as F

df.withColumn("map", F.create_map(*[item for sublist in [[F.lit(x),F.lit(y)]\
                                for x,y in my_dict.items()] for item in sublist]))\
  .filter(' and '.join(["column3.{0}=map.{0}".format(x) for x in my_dict.keys()])+\
          ' and size(column3)=size(map)').drop("map").show()

#+-------+-------+--------------------+
#|column1|column2|             column3|
#+-------+-------+--------------------+
#|      4|      8|[e -> 4.9, b -> 3.7]|
#+-------+-------+--------------------+

+-------+-------+--------------------+
column1|column2| column3|
+-------+-------+--------------------+
 1| 2|[a -> 3.5, b -> 4.2]|
 4| 8|[e -> 4.9, b -> 3.7]|
+-------+-------+--------------------+

+-------+-------+--------------------+
column1|column2| column3|
+-------+-------+--------------------+
 4| 8|[e -> 4.9, b -> 3.7]|
+-------+-------+--------------------+

In [6]:
from pyspark.sql.types import MapType, IntegerType, DoubleType, StringType, StructType, StructField, BooleanType
my_dict = {'b':2.7, 'e':4.9}

from pyspark.sql.functions import udf
def map_equality_comparer(my_dict):
    @udf(BooleanType())
    def comparer(m):
        if len(m) != len(my_dict): return False
        for k, v in m.items():
            if my_dict.get(k) != v: return False
        return True
    return comparer

filtered_df = df.where(map_equality_comparer(my_dict)(df.column3))
filtered_df.show()

+-------+-------+-------+
column1|column2|column3|
+-------+-------+-------+
+-------+-------+-------+

In [7]:
from pyspark.sql.types import MapType, IntegerType, DoubleType, StringType, StructType, StructField
import pyspark.sql.functions as f

schema = StructType([
            StructField('column1', IntegerType()),
            StructField('column2', IntegerType()),
            StructField('column3', MapType(StringType(), DoubleType()))])

data = [(1, 2, {'a':2.0, 'b':4.2}), (4, 8, {'b':3.7, 'e':4.9})]
df = spark.createDataFrame(data, schema=schema)
df.show()

+-------+-------+--------------------+
column1|column2| column3|
+-------+-------+--------------------+
 1| 2|[a -> 2.0, b -> 4.2]|
 4| 8|[e -> 4.9, b -> 3.7]|
+-------+-------+--------------------+

In [8]:
df.filter('column2=column3.a').show()

+-------+-------+--------------------+
column1|column2| column3|
+-------+-------+--------------------+
 1| 2|[a -> 2.0, b -> 4.2]|
+-------+-------+--------------------+

In [9]:
d1 = spark.createDataFrame([("1", [("a","av"),("b","bv")], [("e", "ev"), ("f", "fv")]), \
                                    ("2", [("c", "cv")],  [("g", "gv")])], ["id","list1","list2"])

d1.show()

+---+------------------+------------------+
 id| list1| list2|
+---+------------------+------------------+
 1|[[a, av], [b, bv]]|[[e, ev], [f, fv]]|
 2| [[c, cv]]| [[g, gv]]|
+---+------------------+------------------+

In [10]:
d1.selectExpr("id", "inline_outer(struct(list1,list2))").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o313.selectExpr.
: org.apache.spark.sql.AnalysisException: cannot resolve 'inline(named_struct('list1', `list1`, 'list2', `list2`))' due to data type mismatch: input to function inline should be array of struct type, not struct<list1:array<struct<_1:string,_2:string>>,list2:array<struct<_1:string,_2:string>>>; line 1 pos 0;
'Project [id#2, unresolvedalias(generatorouter(inline(named_struct(list1, list1#3, list2, list2#4))), Some(<function1>))]
+- LogicalRDD [id#2, list1#3, list2#4], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at

In [11]:
list=[['app1'  ,   "anybody love me?"],
['app2'     ,"I hate u"],
['app3'     ,"this hat is good"],
['app4'     ,"I don't like this one"],
['app5'     ,"oh my god"],
['app6'     ,"damn you."],
['app7'     ,"such nice girl"],
['app8'     ,"xxxxx"],
['app9'     ,"pretty prefect"],
['app10'    ,"don't love me"],
['app11'    ,"xxx anybody?"]]

In [12]:
keys=["anybody", "love", "you", "xxx", "don't"]

In [13]:
df=spark.createDataFrame(list,['app','col1'])

df.show()